### Задача
Прогнозирование вероятности возникновения инсульта у человека.

#### Информация об атрибутах
- id: уникальный идентификатор.
- gender: пол: мужской, женский, другое.
- age: возраст пациента.
- hypertension: диагностирована ли у пациента гипертония (0 - нет, 1 - да).
- heart_disease: диагностированы ли у пациента болезни сердца (0 - нет, 1 - да).
- ever married: был ли пациента когда-либо женат/замужем.
- work-type: вид работы (работа с детьми, работа на государственной службе, никогда не работал(а), частная работа, самозанятый(ая)).
- Residence_type: место жительства (сельская местность, город).
- avg_glucose_level: средний уровень глюкозы в крови.
- bmi: значение ИМТ - индекса массы тела.
- smoking_status: отношение к курению (курил(а) в прошлом, никогда не курил(а), курит в настоящий момент, инофрмация неизвестна).
- stroke: таргетный признак, выявлен инсульт или нет (0 - нет, 1 - да). 

In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from scipy import stats
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

### Чтение данных

In [ ]:
dataset = pd.read_csv("./healthcare-dataset-stroke-data.csv")

In [ ]:
dataset.drop('id', axis=1, inplace=True)
dataset

In [ ]:
dataset.info()

### Визуализация данных и вычисление основных характеристик

Возрастное распределение

In [ ]:
plt.hist(dataset['age'].loc[dataset['gender'] == 'Female'], bins=np.arange(15, 90, 5), color='pink', edgecolor='black', label="Female")
plt.hist(dataset['age'].loc[dataset['gender'] == 'Male'], bins=np.arange(15, 90, 5), color='blue', edgecolor='black', label="Male")
plt.hist(dataset['age'].loc[dataset['gender'] == 'Other'], bins=np.arange(15, 90, 5), color='gray', edgecolor='black',  label="Other")
plt.xticks(np.arange(15, 90, 5))
plt.yticks()

plt.xlabel('Age')
plt.ylabel('Count')
plt.legend()
plt.show()

In [ ]:
plt.pie(dataset['gender'].value_counts(), labels=['Female', 'Male', 'Other'], colors = ['pink', 'blue', 'gray'])
plt.show()

Индекс массы тела

In [ ]:
sns.displot(dataset['bmi'])
plt.show()

In [ ]:
sns.heatmap(dataset.corr(numeric_only=True), annot=True, square=True, cmap='coolwarm')
pass

### Обработка пропущенных значений

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset['bmi'] = dataset['bmi'].fillna(dataset['bmi'].median(axis = 0), axis = 0)
dataset.isnull().sum()

### Обработка категориальных признаков

In [ ]:
categorical_columns = [i for i in dataset.columns if dataset[i].dtype == 'object']
numerical_columns = [i for i in dataset.columns if dataset[i].dtype != 'object' and i != 'stroke']
print(categorical_columns)
print(numerical_columns)

In [ ]:
binary_columns = [i for i in categorical_columns if len(dataset[i].unique()) == 2]
nonbinary_columns = [i for i in categorical_columns if len(dataset[i].unique()) > 2]
print(binary_columns, nonbinary_columns)

#### Обработка бинарных категориальных признаков

In [ ]:
print(dataset['Residence_type'].unique(), dataset['ever_married'].unique())

In [ ]:
dataset['Residence_type'] = dataset['Residence_type'].apply(lambda x: 1 if x == 'Urban' else 0)
dataset['ever_married'] = dataset['ever_married'].apply(lambda x: 1 if x == 'Yes' else 0)
dataset

#### Обработка небинарных категориальных признаков

In [ ]:
dataset_nonbinary = pd.get_dummies(dataset[nonbinary_columns])
print(dataset_nonbinary.columns)

### Нормализация

In [ ]:
dataset_numerical = dataset[numerical_columns]
dataset_numerical.describe()

In [ ]:
dataset_numerical = (dataset_numerical - dataset_numerical.mean(axis = 0))/dataset_numerical.std(axis = 0)
dataset_numerical.describe()

In [ ]:
dataset = pd.concat([dataset_numerical, dataset_nonbinary, dataset[binary_columns], dataset['stroke']], axis = 1)
dataset

### Разбивка данных

In [ ]:
X = dataset.drop(['stroke'], axis = 1)
y = dataset['stroke']
X = X.to_numpy()
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 21)

N_train = X_train.shape[0]
N_test = X_test.shape[0]
print(N_train, N_test)

### Запуск классификатора ближайших соседей

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 10)
knn.fit(X_train, y_train)

y_test_predicted = knn.predict(X_test)
err_test = np.mean(y_test_predicted != y_test)

y_train_predict = knn.predict(X_train)
err_train = np.mean(y_train != y_train_predict)

print('err_test=', err_test)
print('err_train=', err_train)

accurancy_train = accuracy_score(y_train, y_train_predict)
accurancy_test = accuracy_score(y_test, y_test_predicted)

print('accurancy_test=', accurancy_test)
print('accurancy_train=', accurancy_train)

### Поиск оптимального значения гиперпараметра (к- ближайших соседей)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
knn = KNeighborsClassifier()

param = np.arange(1, 10)
params = {
    'n_neighbors': param
}

gs = GridSearchCV(knn,params, n_jobs=-1)
gs.fit(X_train, y_train)

In [ ]:
gs.best_params_

### Вычислений ошибок на обучающей и тестовой выборках

In [ ]:
knn = KNeighborsClassifier(**gs.best_params_)

knn.fit(X_train, y_train)

y_test_predicted = knn.predict(X_test)
err_test = np.mean(y_test_predicted != y_test)

y_train_predict = knn.predict(X_train)
err_train = np.mean(y_train != y_train_predict)

print('err_test=', err_test)
print('err_train=', err_train)

accurancy_train = accuracy_score(y_train, y_train_predict)
accurancy_test = accuracy_score(y_test, y_test_predicted)

print('accurancy_test=', accurancy_test)
print('accurancy_train=', accurancy_train)

### RandomForest

In [ ]:
from sklearn import ensemble

In [ ]:
rf = ensemble.RandomForestClassifier(n_estimators = 500, n_jobs=-1)
rf.fit(X_train, y_train)

In [ ]:
y_test_predicted = rf.predict(X_test)
err_test = np.mean(y_test_predicted != y_test)

y_train_predict = rf.predict(X_train)
err_train = np.mean(y_train != y_train_predict)

print('err_test=', err_test)
print('err_train=', err_train)

accurancy_train = accuracy_score(y_train, y_train_predict)
accurancy_test = accuracy_score(y_test, y_test_predicted)

print('accurancy_test=', accurancy_test)
print('accurancy_train=', accurancy_train)

### Выводы

В работе реализовано обучение модели машинного обучения посредством метода k ближайших соседей и Random Forest. 
Наименьшую ошибку показал метод k ближайших соседей со значением параметра = 6.